In [181]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import os
from PIL import Image

# General Election to Parliamentary Constituencies: Trends & Results June-2024

In [157]:
get = requests.get("https://results.eci.gov.in/PcResultGenJune2024/index.htm",)

In [158]:
soup = BeautifulSoup(get._content,'html.parser')

In [172]:
options = soup.find('select').find_all("option")
optionsData = []
for o in options:
    optionsData.append(o.get("value"))
optionsData = optionsData[1:]

In [167]:
table = soup.find("table")
table = pd.read_html(str(table))[0]

C:\Users\yahya\AppData\Local\Temp\ipykernel_7684\703268541.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table))[0]


In [185]:
json.dumps(table.to_dict())

'{"Party": {"0": "Bharatiya Janata Party - BJP", "1": "Indian National Congress - INC", "2": "Samajwadi Party - SP", "3": "All India Trinamool Congress - AITC", "4": "Dravida Munnetra Kazhagam - DMK", "5": "Telugu Desam - TDP", "6": "Janata Dal (United) - JD(U)", "7": "Shiv Sena (Uddhav Balasaheb Thackrey) - SHSUBT", "8": "Nationalist Congress Party \\u2013 Sharadchandra Pawar - NCPSP", "9": "Shiv Sena - SHS", "10": "Lok Janshakti Party(Ram Vilas) - LJPRV", "11": "Yuvajana Sramika Rythu Congress Party - YSRCP", "12": "Rashtriya Janata Dal - RJD", "13": "Communist Party of India (Marxist) - CPI(M)", "14": "Indian Union Muslim League - IUML", "15": "Aam Aadmi Party - AAAP", "16": "Jharkhand Mukti Morcha - JMM", "17": "Janasena Party - JnP", "18": "Communist Party of India (Marxist-Leninist) (Liberation) - CPI(ML)(L)", "19": "Janata Dal (Secular) - JD(S)", "20": "Viduthalai Chiruthaigal Katchi - VCK", "21": "Communist Party of India - CPI", "22": "Rashtriya Lok Dal - RLD", "23": "Jammu & 

In [189]:
with open("./Parliamentary Constituencies/main.json",'w') as mainf:
    mainf.write(json.dumps(table.to_dict())) 

# Inside

In [193]:
main = "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U01.htm"
data = requests.get(main)
soup = BeautifulSoup(data.text,"html.parser")

In [195]:
title = soup.find("div","page-title").find("strong").text.strip()
title

'Andaman & Nicobar Islands'

In [196]:
outTabel = pd.read_html(str(soup.find("table")))[0]
outTabel

C:\Users\yahya\AppData\Local\Temp\ipykernel_7684\605321805.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  outTabel = pd.read_html(str(soup.find("table")))[0]


,Party,Won,Leading,Total
0,Bharatiya Janata Party - BJP,1,0,1
1,Total,1,0,1


In [143]:
os.makedirs(f"./Parliamentary Constituencies/{title}",exist_ok=True)
with open(f"./Parliamentary Constituencies/{title}/data.json",'w') as ff:
    ff.write(json.dumps(json.loads(outTabel.to_json()),indent=4))

In [144]:
allConsituency = soup.find("select").find_all("option")[1:]
allConsituencyData = []
for d2 in allConsituency:
    allConsituencyData.append(d2.get("value"))

In [199]:
ConsituencyWise = requests.get(f"https://results.eci.gov.in/PcResultGenJune2024/Constituencywise{allConsituencyData[0]}.htm")

In [200]:
soupIn = BeautifulSoup(ConsituencyWise.text,"html.parser")

In [201]:
ConsituencyTitle = soupIn.find("h2").find("span").text.strip()

In [202]:
ConsituencyTable =  pd.read_html(str(soupIn.find("table")))[0]

C:\Users\yahya\AppData\Local\Temp\ipykernel_7684\3052624219.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  ConsituencyTable =  pd.read_html(str(soupIn.find("table")))[0]


In [207]:
ConsituencyData = ConsituencyTable.iloc[:-1,:].to_dict()

In [204]:
ConsituencyTotal = ConsituencyTable.iloc[-1].dropna().to_dict()

In [218]:
with open(f"./Parliamentary Constituencies/{title}/{ConsituencyTitle}.json",'w') as ff:
    ff.write(json.dumps(ConsituencyData))

### Extraction of all Data

In [223]:
for opd in optionsData:
    main = f"https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-{opd}.htm"
    data = requests.get(main)
    soup = BeautifulSoup(data.text,"html.parser")
    title = soup.find("div","page-title").find("strong").text.strip()
    outTabel = pd.read_html(str(soup.find("table")))[0]
    os.makedirs(f"./Parliamentary Constituencies/{title}",exist_ok=True)
    with open(f"./Parliamentary Constituencies/{title}/data.json",'w') as ff:
        ff.write(json.dumps(json.loads(outTabel.to_json()),indent=4))
    allConsituency = soup.find("select").find_all("option")[1:]
    allConsituencyData = []
    for d2 in allConsituency:
        allConsituencyData.append(d2.get("value"))
    for d3 in allConsituencyData:
        ConsituencyWise = requests.get(f"https://results.eci.gov.in/PcResultGenJune2024/Constituencywise{d3}.htm")
        soupIn = BeautifulSoup(ConsituencyWise.text,"html.parser")
        ConsituencyTitle = soupIn.find("h2").find("span").text.strip()
        ConsituencyTable =  pd.read_html(str(soupIn.find("table")))[0]
        ConsituencyData = ConsituencyTable.iloc[:-1,:].to_dict()
        ConsituencyTotal = ConsituencyTable.iloc[-1].dropna().to_dict()
        ConsituencyData['ConsituencyTotal'] = ConsituencyTotal
        with open(f"./Parliamentary Constituencies/{title}/{ConsituencyTitle}.json",'w') as ff:
            ff.write(json.dumps(ConsituencyData))

C:\Users\yahya\AppData\Local\Temp\ipykernel_7684\4232467316.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  outTabel = pd.read_html(str(soup.find("table")))[0]
C:\Users\yahya\AppData\Local\Temp\ipykernel_7684\4232467316.py:18: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  ConsituencyTable =  pd.read_html(str(soupIn.find("table")))[0]
C:\Users\yahya\AppData\Local\Temp\ipykernel_7684\4232467316.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  outTabel = pd.read_html(str(soup.find("table")))[0]
C:\Users\yahya\AppData\Local\Temp\ipykernel_7684\4232467316.py:18: FutureWarning: Passing literal html to 'read_html' is 

# Assembly Constituency General
### Just add the links to Extract

In [281]:
# Andra Pradesh
# get = requests.get("https://results.eci.gov.in/AcResultGenJune2024/partywiseresult-S01.htm")
# Odisha
get = requests.get("https://results.eci.gov.in/AcResultGenJune2024/partywiseresult-S18.htm")

In [282]:
soup = BeautifulSoup(get._content,'html.parser')
options = soup.find('select').find_all("option")
optionsData = []
for o in options:
    optionsData.append(o.get("value"))
optionsData = optionsData[1:]
title = soup.find("strong").text.strip()
table = soup.find("table")
table = pd.read_html(str(table))[0]

C:\Users\yahya\AppData\Local\Temp\ipykernel_7684\1049604810.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(table))[0]


In [283]:
os.makedirs(f"./Assembly Constituencies/{title}")
with open(f"./Assembly Constituencies/{title}/main.json",'w') as ifile:
    ifile.write(json.dumps(table.to_dict()))

In [284]:
for idata in optionsData:
    iget = requests.get(f"https://results.eci.gov.in/AcResultGenJune2024/Constituencywise{idata}.htm")
    isoup = BeautifulSoup(iget.text,"html.parser")
    titleIn = isoup.find("h2").find("span").text.strip()
    tableIn = pd.read_html(str(isoup.find("table")))[0]
    DataIn = tableIn.iloc[:-1,:].to_dict()
    totalIn = tableIn.iloc[-1].dropna().to_dict()
    DataIn['Total'] = totalIn
    with open(f"./Assembly Constituencies/{title}/{titleIn}.json",'w') as ifile:
        ifile.write(json.dumps(DataIn))

C:\Users\yahya\AppData\Local\Temp\ipykernel_7684\562563853.py:5: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tableIn = pd.read_html(str(isoup.find("table")))[0]
C:\Users\yahya\AppData\Local\Temp\ipykernel_7684\562563853.py:5: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tableIn = pd.read_html(str(isoup.find("table")))[0]
C:\Users\yahya\AppData\Local\Temp\ipykernel_7684\562563853.py:5: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tableIn = pd.read_html(str(isoup.find("table")))[0]
C:\Users\yahya\AppData\Local\Temp\ipykernel_7684\562563853.py:5: FutureWarning: Passing literal html to 'read_html' is deprecated and wi

## General Election to Assembly Constituencies: Trends & Results June-2024
#### Inside AP and Odisha All Constituencies glance

In [285]:
# AP
get = requests.get("https://results.eci.gov.in/AcResultGenJune2024/statewiseS011.htm")

In [306]:
jsoup = BeautifulSoup(get.text,'html.parser')
jtable =jsoup.find("table")
jtable.find("tr").decompose()

In [312]:
jtable

<table class="table table-striped table-bordered">
<thead>

<tr>
<th style="text-align:left">Constituency  </th>
<th style="text-align:left">Const. No.</th>
<th style="text-align:left">Leading Candidate </th>
<th style="text-align:left">Leading Party </th>
<th style="text-align:left">Trailing Candidate </th>
<th style="text-align:left">Trailing Party </th>
<th style="text-align:left">Margin </th>
<th style="text-align:left">Round</th>
<th style="text-align:left">Status</th>
</tr>
</thead>
<tbody>
<tr><td align="left">Achanta</td><td align="right">56</td><td align="left">SATYANARAYANA PITHANI</td><td align="left"><table><tbody><tr><td align="left">Telugu Desam</td><td><span class="tooltip-icon" style="display:block">i</span><div class="tooltip"><h3 class="tooltiptext">Party Wise State Trends</h3><table class="table table-striped"><tbody><tr><td>Leading In</td><td>:</td><td>0</td></tr><tr><td>Won In</td><td>:</td><td>135</td></tr><tr><td>Trailing In</td><td>:</td><td>0</td></tr></tbody><

In [311]:
pd.read_html(str(jtable))[0]

C:\Users\yahya\AppData\Local\Temp\ipykernel_7684\1329160196.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  pd.read_html(str(jtable))[0]


,Constituency,Const. No.,Leading Candidate,Leading Party,Trailing Candidate,Trailing Party,Margin,Round,Status
0,Achanta,56,SATYANARAYANA PITHANI,Telugu DesamiParty Wise State TrendsLeading In...,CHERUKUVADA SRI RANGANADHA RAJU,Yuvajana Sramika Rythu Congress Party iParty W...,26554.0,14/14,Result Declared
1,Telugu Desam,iParty Wise State TrendsLeading In:0Won In:135...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Leading In,:,0,NaN,NaN,NaN,NaN,NaN,NaN
3,Won In,:,135,NaN,NaN,NaN,NaN,NaN,NaN
4,Trailing In,:,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
176,Yuvajana Sramika Rythu Congress Party,iParty Wise State TrendsLeading In:0Won In:11T...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
177,Leading In,:,0,NaN,NaN,NaN,NaN,NaN,NaN
178,Won In,:,11,NaN,NaN,NaN,NaN,NaN,NaN
179,Trailing In,:,0,NaN,NaN,NaN,NaN,NaN,NaN


# Bye Election to Assembly Constituencies: Results June-2024

In [313]:
get = requests.get("https://results.eci.gov.in/AcResultByeJune2024/index.htm")
ksoup = BeautifulSoup(get.text,'html.parser')

In [341]:
continers = ksoup.find("div","row justify-content-center").find_all("div","col-md-3 col-12")
ktitle = continers[0].find("h3").text.strip()
kstate = continers[0].find("h4").text.strip()
kname = continers[0].find("h5").text.strip()
kby = continers[0].find("h6").text.strip()

In [348]:
os.makedirs(f"./Bye Election to Assembly Constituencies")
for con in  continers:
    link = con.find("a")['href'].split("-")[1]
    ktitle = con.find("h3").text.strip()
    kstate = con.find("h4").text.strip()
    kname = con.find("h5").text.strip()
    kby = con.find("h6").text.strip()
    kget = requests.get(f"https://results.eci.gov.in/AcResultByeJune2024/Constituencywise{link}")
    kisoup = BeautifulSoup(kget.text,'html.parser')
    kDataIn = {}
    ktableIn = pd.read_html(str(kisoup.find("table")))[0]
    kDataIn['Table'] = ktableIn.iloc[:-1,:].to_dict()
    ktotalIn = ktableIn.iloc[-1].dropna().to_dict()
    kDataIn['Total'] = ktotalIn
    kDataIn['Name'] = kname
    kDataIn['State'] = kstate
    kDataIn['Title'] = ktitle
    kDataIn['By'] = kby
    with open(f"./Bye Election to Assembly Constituencies/{ktitle}.json",'w') as kf:
        kf.write(json.dumps(kDataIn))

C:\Users\yahya\AppData\Local\Temp\ipykernel_7684\2814153244.py:11: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  ktableIn = pd.read_html(str(kisoup.find("table")))[0]
C:\Users\yahya\AppData\Local\Temp\ipykernel_7684\2814153244.py:11: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  ktableIn = pd.read_html(str(kisoup.find("table")))[0]
C:\Users\yahya\AppData\Local\Temp\ipykernel_7684\2814153244.py:11: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  ktableIn = pd.read_html(str(kisoup.find("table")))[0]
C:\Users\yahya\AppData\Local\Temp\ipykernel_7684\2814153244.py:11: FutureWarning: Passing literal html to 'read_html' is dep